In [4]:
import torch
import torchvision
import time
torch.cuda.is_available()##判断是否有cuda

###注意 这里其实除了数据其他不需要再赋值了 直接net.to(device)就可以了 
def is_cuda(x,use_cuda=True):
    if use_cuda:
        return x.cuda() if torch.cuda.is_available() else x
    else:
        return x
    
import torch
import torch.nn as nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=64 * 4 * 4, out_features=10)
        )

    def forward(self, x):
        out = self.sequential(x)
        return out

In [5]:
###原始数据集
rootdir=r'D:\code_file\dplearning_xiaotudui\data'
train_datasets=torchvision.datasets.CIFAR10(rootdir,transform=torchvision.transforms.ToTensor(),train=True)
test_datasets=torchvision.datasets.CIFAR10(rootdir,transform=torchvision.transforms.ToTensor(),train=False)
train_size=len(train_datasets)
test_size=len(test_datasets)
print('训练集数据大小{},测试集数据大小{}'.format(train_size,test_size))
###数据集加载
train_dataloader=torch.utils.data.DataLoader(train_datasets,batch_size=64,shuffle=True)
test_dataloader=torch.utils.data.DataLoader(test_datasets,batch_size=64,shuffle=True)
###模型加载
net=Net()
###这样判断 有cuda 则进行cuda运算更合理
net=is_cuda(net)


###定义loss function
loss_func=nn.CrossEntropyLoss()
loss_func=is_cuda(loss_func)


###定义优化器
optimz=torch.optim.SGD(net.parameters(),momentum=0.9)


训练集数据大小50000,测试集数据大小10000


In [6]:
epoch=10
train_step=0
test_step=0
start_time=time.time()
for i in range(1,epoch+1):
    for datas in train_dataloader:
        optimz.zero_grad()
        data,target=datas
        ###数据上cuda
        data=is_cuda(data,use_cuda=True)
        target=is_cuda(target,use_cuda=True)
        
        output=net(data)
        loss=loss_func(output,target)
        loss.backward()
        optimz.step()
        if train_step%100==0:
            end_time=time.time()
            print(end_time-start_time)
            print(end_time-start_time)
            print('训练第{}步，训练集上的损失为{}'.format(train_step,loss))
        train_step+=1
    with torch.no_grad():
        total_loss=0
        total_accuracy=0
        for datas in test_dataloader:
            data,target=datas
            ###转入cuda
            data=is_cuda(data,use_cuda=True)
            target=is_cuda(target,use_cuda=True)
            
            output=net(data)
            loss=loss_func(output,target)
            total_loss+=loss
            total_accuracy+=(output.argmax(1)==target).sum()
        
        print('测试集上的梯度{}'.format(total_loss))
        print('测试集上的准确率{}'.format(total_accuracy/test_size))

0.04743671417236328
0.04743671417236328
训练第0步，训练集上的损失为2.309211254119873
3.539426326751709
3.539426326751709
训练第100步，训练集上的损失为2.280055284500122
6.860842704772949
6.860842704772949
训练第200步，训练集上的损失为2.2650651931762695
10.161633014678955
10.161633014678955
训练第300步，训练集上的损失为2.121889114379883
13.494877576828003
13.494877576828003
训练第400步，训练集上的损失为1.993196725845337
16.902547121047974
16.902547121047974
训练第500步，训练集上的损失为2.0427794456481934
20.266924381256104
20.266924381256104
训练第600步，训练集上的损失为1.8557003736495972
23.689820528030396
23.689820528030396
训练第700步，训练集上的损失为1.8310956954956055
测试集上的梯度283.2325439453125
测试集上的准确率0.36640000343322754
30.314990282058716
30.314990282058716
训练第800步，训练集上的损失为1.8817858695983887
33.75321817398071
33.75321817398071
训练第900步，训练集上的损失为1.7570765018463135
37.253735303878784
37.253735303878784
训练第1000步，训练集上的损失为1.6986826658248901
40.63629865646362
40.63629865646362
训练第1100步，训练集上的损失为1.5896015167236328
44.08307075500488
44.08307075500488
训练第1200步，训练集上的损失为1.6320250034332275
47.611043

KeyboardInterrupt: 

In [14]:
####使用cpu或gpu计算 定义设备
device1=torch.device('cpu')
###如果有多个cuda 可以torch.device('cuda:0')代表第一个gpu
device2=torch.device('cuda')
###然后可以把变量转移到设备上
net=net.to(device1)

device(type='cuda')